<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_preprocessing/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#talib 설치
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null
!tar xvzf ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null

import os

os.chdir('ta-lib')  

!./configure --prefix=/usr 2>&1 > /dev/null
!make 2>&1 > /dev/null
!make install 2>&1 > /dev/null

os.chdir('../')

!pip install TA-Lib 2>&1 > /dev/null

./configure: line 4354: /usr/bin/file: No such file or directory
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
libtool: link: warning: `-version-info/-version-number' is ignored for convenience libraries
ar: `u' modifier ignored since `D' is the default (see `U')
ar: `u' modifier ignored since `D' is the default (see `U')
gen_code.c: In function ‘printFuncHeaderDoc’:
gen_code.c:3456:4: warning: format not a string literal and no format arguments [-Wformat-security]
    fprintf( out, prefix );
    ^~~~~~~


In [2]:
import re
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive
# import talib
import numpy as np

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Mounted at /content/drive


In [3]:
# 저장된 삼성전자(KR7005930003) 데이터 가져오기

with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) 

df = pd.read_json(json_data['KR7005930003'], orient ='index') 
samsung = df.transpose()
samsung

,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,EPS,PER,BPS,PBR,DPS,DVD_YLD,ACC_TRDVOL
0,2021/11/09,005930,삼성전자,"70,500","70,300","71,000","70,100","420,869,669,775,000","3,841",18.35,"39,406",1.79,"2,994",4.25,None
1,2021/11/08,005930,삼성전자,"70,600","70,400","70,600","69,800","421,466,648,030,000","3,841",18.38,"39,406",1.79,"2,994",4.24,None
2,2021/11/05,005930,삼성전자,"70,200","71,600","71,600","70,200","419,078,735,010,000","3,841",18.28,"39,406",1.78,"2,994",4.26,"12,667,743"
3,2021/11/04,005930,삼성전자,"70,600","71,200","71,600","70,500","421,466,648,030,000","3,841",18.38,"39,406",1.79,"2,994",4.24,"11,818,987"
4,2021/11/03,005930,삼성전자,"70,400","71,700","71,700","70,100","420,272,691,520,000","3,841",18.33,"39,406",1.79,"2,994",4.25,"12,770,428"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6713,1995/05/08,005930,삼성전자,"121,000","122,000","122,000","120,500","6,578,606,045,000",-,-,-,-,-,-,"91,810"
6714,1995/05/06,005930,삼성전자,"122,000","122,000","123,500","122,000","6,632,974,690,000",-,-,-,-,-,-,"47,440"
6715,1995/05/04,005930,삼성전자,"122,500","124,000","124,500","122,000","6,660,159,012,500",-,-,-,-,-,-,"175,590"
6716,1995/05/03,005930,삼성전자,"123,500","119,500","126,500","119,500","6,714,527,657,500",-,-,-,-,-,-,"382,980"


In [4]:
#시간순 재정렬
samsung = samsung.sort_values(by=['TRD_DD'])
samsung.reset_index(drop=True,inplace=True)
samsung['TRD_DD']=pd.to_datetime(samsung['TRD_DD']) #datetime변환

In [5]:
# 일년치 데이터만 추출
samsung01 = samsung.copy()
# samsung01 = samsung01[(samsung01['TRD_DD']>='2020-10-21') & (samsung01['TRD_DD']<='2021-10-20')]
samsung01 = samsung01.reset_index(drop = True)
samsung01

,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,EPS,PER,BPS,PBR,DPS,DVD_YLD,ACC_TRDVOL
0,1995-05-02,005930,삼성전자,"119,500","121,000","121,000","118,500","6,497,053,077,500",-,-,-,-,-,-,"139,560"
1,1995-05-03,005930,삼성전자,"123,500","119,500","126,500","119,500","6,714,527,657,500",-,-,-,-,-,-,"382,980"
2,1995-05-04,005930,삼성전자,"122,500","124,000","124,500","122,000","6,660,159,012,500",-,-,-,-,-,-,"175,590"
3,1995-05-06,005930,삼성전자,"122,000","122,000","123,500","122,000","6,632,974,690,000",-,-,-,-,-,-,"47,440"
4,1995-05-08,005930,삼성전자,"121,000","122,000","122,000","120,500","6,578,606,045,000",-,-,-,-,-,-,"91,810"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6713,2021-11-03,005930,삼성전자,"70,400","71,700","71,700","70,100","420,272,691,520,000","3,841",18.33,"39,406",1.79,"2,994",4.25,"12,770,428"
6714,2021-11-04,005930,삼성전자,"70,600","71,200","71,600","70,500","421,466,648,030,000","3,841",18.38,"39,406",1.79,"2,994",4.24,"11,818,987"
6715,2021-11-05,005930,삼성전자,"70,200","71,600","71,600","70,200","419,078,735,010,000","3,841",18.28,"39,406",1.78,"2,994",4.26,"12,667,743"
6716,2021-11-08,005930,삼성전자,"70,600","70,400","70,600","69,800","421,466,648,030,000","3,841",18.38,"39,406",1.79,"2,994",4.24,None


In [7]:
#인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
samsung_bt = samsung01[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL']]
samsung_bt['TRD_DD'] = pd.to_datetime(samsung_bt['TRD_DD'])
samsung_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume'}, inplace=True)
samsung_bt.set_index('Date',drop=True,inplace=True)
samsung_bt['OpenInterest'] =  np.zeros(6718, dtype=int)
samsung_bt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
1995-05-02,"121,000","121,000","118,500","119,500","139,560",0
1995-05-03,"119,500","126,500","119,500","123,500","382,980",0
1995-05-04,"124,000","124,500","122,000","122,500","175,590",0
1995-05-06,"122,000","123,500","122,000","122,000","47,440",0
1995-05-08,"122,000","122,000","120,500","121,000","91,810",0
...,...,...,...,...,...,...
2021-11-03,"71,700","71,700","70,100","70,400","12,770,428",0
2021-11-04,"71,200","71,600","70,500","70,600","11,818,987",0
2021-11-05,"71,600","71,600","70,200","70,200","12,667,743",0


In [8]:
#데이터프레임 콤마(,) 제거 그리고 타입 소수로 변환
samsung_bt['Open'] = samsung_bt['Open'].str.replace(',','').astype('float')
samsung_bt['High'] = samsung_bt['High'].str.replace(',','').astype('float')
samsung_bt['Low'] = samsung_bt['Low'].str.replace(',','').astype('float')
samsung_bt['Close'] = samsung_bt['Close'].str.replace(',','').astype('float')
samsung_bt['Volume'] = samsung_bt['Volume'].str.replace(',','').astype('float')
samsung_bt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
1995-05-02,121000.0,121000.0,118500.0,119500.0,139560.0,0
1995-05-03,119500.0,126500.0,119500.0,123500.0,382980.0,0
1995-05-04,124000.0,124500.0,122000.0,122500.0,175590.0,0
1995-05-06,122000.0,123500.0,122000.0,122000.0,47440.0,0
1995-05-08,122000.0,122000.0,120500.0,121000.0,91810.0,0
...,...,...,...,...,...,...
2021-11-03,71700.0,71700.0,70100.0,70400.0,12770428.0,0
2021-11-04,71200.0,71600.0,70500.0,70600.0,11818987.0,0
2021-11-05,71600.0,71600.0,70200.0,70200.0,12667743.0,0


In [9]:
samsung_bt = samsung_bt.replace(0, -1)
samsung_bt

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
1995-05-02,121000.0,121000.0,118500.0,119500.0,139560.0,-1
1995-05-03,119500.0,126500.0,119500.0,123500.0,382980.0,-1
1995-05-04,124000.0,124500.0,122000.0,122500.0,175590.0,-1
1995-05-06,122000.0,123500.0,122000.0,122000.0,47440.0,-1
1995-05-08,122000.0,122000.0,120500.0,121000.0,91810.0,-1
...,...,...,...,...,...,...
2021-11-03,71700.0,71700.0,70100.0,70400.0,12770428.0,-1
2021-11-04,71200.0,71600.0,70500.0,70600.0,11818987.0,-1
2021-11-05,71600.0,71600.0,70200.0,70200.0,12667743.0,-1


In [10]:
!pip install backtrader

     |████████████████████████████████| 410 kB 25.4 MB/s 


In [11]:
import backtrader as bt
from datetime import datetime

# time period 를 5~25로 설정하고 실행

for i in range(5, 26):
    try:
        class firstStrategy(bt.Strategy):
          def __init__(self):
            self.rsi = bt.indicators.RSI_SMA(self.data.close, period = i)

          def next(self):
            if not self.position:
              if self.rsi < 30:
                  self.buy(size=100)
              else:
                if self.rsi > 70:
                  self.sell(size=100)


        #초기 금액 설정
        startcash = 10000

        #Cerebro 인스턴스 생성
        cerebro = bt.Cerebro()

        #Cerebro 에 설정한 전략 추가
        cerebro.addstrategy(firstStrategy)

        #데이터 가져오기
        data = bt.feeds.PandasData(dataname=samsung_bt)
        #data = bt.feeds.YahooFinanceData(dataname='MSFT', fromdate=datetime(2011, 1, 1), todate=datetime(2012, 12, 31))
        #데이터를 Cerebro에 추가
        cerebro.adddata(data)

        #초기금액 세팅
        cerebro.broker.setcash(startcash)

        #Cerebro실행 (전략, 데이터 추가한 후)
        cerebro.run()

        #결과(전략을 통해 얻을수 있는 금액)
        portvalue = cerebro.broker.getvalue()

        #차익계산
        pnl = portvalue - startcash

        # 수익률 계산
        earning = ((portvalue - startcash) / startcash) * 100

        #결과출력
        print('time period: {}'.format(i))
        print('Final Portfolio Value: ${}'.format(portvalue))
        print('P/L: ${}'.format(pnl))
        print('earning rates: {}%'.format(earning))
        print('------------------------------------------------------')

        #차트출력
        # cerebro.plot(style='candlestick')
    except:
        print(i, 'error')
        print('------------------------------------------------------')

5 error
------------------------------------------------------
6 error
------------------------------------------------------
7 error
------------------------------------------------------
8 error
------------------------------------------------------
9 error
------------------------------------------------------
10 error
------------------------------------------------------
11 error
------------------------------------------------------
12 error
------------------------------------------------------
time period: 13
Final Portfolio Value: $4960000.0
P/L: $4950000.0
earning rates: 49500.0%
------------------------------------------------------
time period: 14
Final Portfolio Value: $4960000.0
P/L: $4950000.0
earning rates: 49500.0%
------------------------------------------------------
time period: 15
Final Portfolio Value: $4810000.0
P/L: $4800000.0
earning rates: 48000.0%
------------------------------------------------------
time period: 16
Final Portfolio Value: $4910000.0
P/L: $49

# GA

In [12]:
df = samsung_bt.copy()
df = df.drop(['OpenInterest'], axis = 1)
df

,Open,High,Low,Close,Volume
Date,,,,,
1995-05-02,121000.0,121000.0,118500.0,119500.0,139560.0
1995-05-03,119500.0,126500.0,119500.0,123500.0,382980.0
1995-05-04,124000.0,124500.0,122000.0,122500.0,175590.0
1995-05-06,122000.0,123500.0,122000.0,122000.0,47440.0
1995-05-08,122000.0,122000.0,120500.0,121000.0,91810.0
...,...,...,...,...,...
2021-11-03,71700.0,71700.0,70100.0,70400.0,12770428.0
2021-11-04,71200.0,71600.0,70500.0,70600.0,11818987.0
2021-11-05,71600.0,71600.0,70200.0,70200.0,12667743.0


In [13]:
import backtrader as bt
class CrossoverStrategy(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(fast_period=12, slow_period=26, signal_period=9)

    def __init__(self):

        self.fast_ma = bt.indicators.EMA(self.data.close, period=self.p.fast_period)
        self.slow_ma = bt.indicators.EMA(self.data.close, period=self.p.slow_period)
        self.macd_line = self.fast_ma - self.slow_ma
        self.signal_line = bt.indicators.EMA(self.macd_line, period=self.p.signal_period)
        self.macd_crossover = bt.indicators.CrossOver(self.macd_line, self.signal_line)

    def next(self):

        if self.macd_crossover > 0:
            self.buy()  # enter long position
        elif self.macd_crossover < 0:
            self.close()  # close long position

In [14]:
TICKER = "NVDA"
STRATEGY_PARAMS = dict(fast_period=12, slow_period=26, signal_period=9)


def run_backtest(plot=True, **strategy_params):

    cerebro = bt.Cerebro()
    cerebro.adddata(
        bt.feeds.PandasData(dataname=df, name=TICKER)
    )

    # Remember to set it high enough or the strategy may not
    # be able to trade because of short of cash
    cerebro.broker.setcash(10000.0)

    # Print out the starting conditions
    print(f"Starting Portfolio Value: {cerebro.broker.getvalue():,.2f}")

    # Although we have defined some default params in the strategy,
    # we can override it by passing in keyword arguments here.
    cerebro.addstrategy(CrossoverStrategy, **strategy_params)

    cerebro.addobserver(bt.observers.Trades)
    cerebro.addobserver(bt.observers.DrawDown)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio)

    # Let's say that we have 0.25% slippage and commission per trade,
    # that is 0.5% in total for a round trip.
    cerebro.broker.setcommission(commission=0.0025, margin=False)

    # Run over everything
    strats = cerebro.run()

    print(f"Final Portfolio Value:    {cerebro.broker.getvalue():,.2f}")

    if plot:
        figure = cerebro.plot(volume=False, width=25, height=25)[0][0] 
        figure.savefig('example_sma.png') # 코랩에서 시각화하는 방법

In [15]:
run_backtest(plot=True, **STRATEGY_PARAMS)

Starting Portfolio Value: 10,000.00
Final Portfolio Value:    10,000.00


<IPython.core.display.Javascript object>

In [17]:
!pip install deap

     |████████████████████████████████| 160 kB 40.6 MB/s 


In [19]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import trange
import time
import random
from deap import base
from deap import creator
from deap import tools


# fix the seed so that we will get the same results
# feel free to change it or comment out the line
random.seed(1)

# GA parameters
PARAM_NAMES = ["fast_period", "slow_period", "signal_period"]
NGEN = 20
NPOP = 100
CXPB = 0.5
MUTPB = 0.3

data = bt.feeds.PandasData(dataname=df)


def evaluate(individual, plot=False, log=False):

    # convert list of parameter values into dictionary of kwargs
    strategy_params = {k: v for k, v in zip(PARAM_NAMES, individual)}

    # fast moving average by definition cannot be slower than the slow one
    if strategy_params["fast_period"] >= strategy_params["slow_period"]:
        return [-np.inf]

    # by setting stdstats to False, backtrader will not store the changes in
    # statistics like number of trades, buys & sells, etc.
    cerebro = bt.Cerebro(stdstats=False)
    cerebro.adddata(data)

    # Remember to set it high enough or the strategy may not
    # be able to trade because of short of cash
    initial_capital = 10_000.0
    cerebro.broker.setcash(initial_capital)

    # Pass in the genes of the individual as kwargs
    cerebro.addstrategy(CrossoverStrategy, **strategy_params)

    # This is needed for calculating our fitness score
    cerebro.addanalyzer(bt.analyzers.DrawDown)

    # Let's say that we have 0.25% slippage and commission per trade,
    # that is 0.5% in total for a round trip.
    cerebro.broker.setcommission(commission=0.0025, margin=False)

    # Run over everything
    strats = cerebro.run()

    profit = cerebro.broker.getvalue() - initial_capital
    max_dd = strats[0].analyzers.drawdown.get_analysis()["max"]["moneydown"]
    fitness = profit / (max_dd if max_dd > 0 else 1)

    if log:
        print(f"Starting Portfolio Value: {initial_capital:,.2f}")
        print(f"Final Portfolio Value:    {cerebro.broker.getvalue():,.2f}")
        print(f"Total Profit:             {profit:,.2f}")
        print(f"Maximum Drawdown:         {max_dd:,.2f}")
        print(f"Profit / Max DD:          {fitness}")

    if plot:
        cerebro.plot()

    return [fitness]


# our fitness score is supposed to be maximised and there is only 1 objective
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# our individual is a list of genes, with the fitness score the higher the better
creator.create("Individual", list, fitness=creator.FitnessMax)

# register some handy functions for calling
toolbox = base.Toolbox()
toolbox.register("indices", random.sample, range(NPOP), NPOP)
# crossover strategy
toolbox.register("mate", tools.cxUniform, indpb=CXPB)
# mutation strategy
toolbox.register("mutate", tools.mutUniformInt, low=1, up=151, indpb=0.2)
# selection strategy
toolbox.register("select", tools.selTournament, tournsize=3)
# fitness function
toolbox.register("evaluate", evaluate)

# definition of an individual & a population
toolbox.register("attr_fast_period", random.randint, 1, 51)
toolbox.register("attr_slow_period", random.randint, 10, 151)
toolbox.register("attr_signal_period", random.randint, 1, 101)
toolbox.register(
    "individual",
    tools.initCycle,
    creator.Individual,
    (
        toolbox.attr_fast_period,
        toolbox.attr_slow_period,
        toolbox.attr_signal_period,
    ),
)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

mean = np.ndarray(NGEN)
best = np.ndarray(NGEN)
hall_of_fame = tools.HallOfFame(maxsize=3)

t = time.perf_counter()
pop = toolbox.population(n=NPOP)
for g in trange(NGEN):
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))

    # Apply crossover on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    # Apply mutation on the offspring
    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    # The population is entirely replaced by the offspring
    pop[:] = offspring
    hall_of_fame.update(pop)
    print(
        "HALL OF FAME:\n"
        + "\n".join(
            [
                f"    {_}: {ind}, Fitness: {ind.fitness.values[0]}"
                for _, ind in enumerate(hall_of_fame)
            ]
        )
    )

    fitnesses = [
        ind.fitness.values[0] for ind in pop if not np.isinf(ind.fitness.values[0])
    ]
    mean[g] = np.mean(fitnesses)
    best[g] = np.max(fitnesses)

end_t = time.perf_counter()
print(f"Time Elapsed: {end_t - t:,.2f}")

fig, ax = plt.subplots(sharex=True, figsize=(16, 9))

sns.lineplot(x=range(NGEN), y=mean, ax=ax, label="Average Fitness Score")
sns.lineplot(x=range(NGEN), y=best, ax=ax, label="Best Fitness Score")
ax.set_title("Fitness Score")
ax.set_xticks(range(NGEN))
ax.set_xlabel("Iteration")

plt.tight_layout()
plt.show()

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
  5%|▌         | 1/20 [03:45<1:11:19, 225.25s/it]

HALL OF FAME:
    0: [7, 74, 94], Fitness: 0.0
    1: [35, 70, 9], Fitness: 0.0
    2: [3, 51, 58], Fitness: 0.0


 10%|█         | 2/20 [06:19<55:02, 183.50s/it]  

HALL OF FAME:
    0: [7, 74, 94], Fitness: 0.0
    1: [35, 70, 9], Fitness: 0.0
    2: [3, 51, 58], Fitness: 0.0


 15%|█▌        | 3/20 [09:10<50:19, 177.60s/it]

...

 95%|█████████▌| 19/20 [49:32<02:33, 153.82s/it]

HALL OF FAME:
    0: [7, 74, 94], Fitness: 0.0
    1: [35, 70, 9], Fitness: 0.0
    2: [3, 51, 58], Fitness: 0.0


100%|██████████| 20/20 [51:49<00:00, 155.47s/it]

HALL OF FAME:
    0: [7, 74, 94], Fitness: 0.0
    1: [35, 70, 9], Fitness: 0.0
    2: [3, 51, 58], Fitness: 0.0
Time Elapsed: 3,109.34


<IPython.core.display.Javascript object>